Промаркируем топиками все единицы векторной базы

Инсталлы

In [1]:
! pip install --upgrade --quiet yandex-cloud-ml-sdk


Импорты

In [14]:
import os
from yandex_cloud_ml_sdk import YCloudML
import json
import pandas as pd
import ast


Функция для запросов к модели

In [15]:


folder_id = os.environ["FOLDER_ID"]
api_key = os.environ["API_KEY"]

sdk = YCloudML(folder_id=folder_id, auth=api_key)
model = sdk.models.completions("yandexgpt", model_version="rc")


valid_topics = 'проходной балл, меганаправления, количество бюджетных мест, цена года обучения, общежития, индивидуальные достижения, компанни-партнеры, стажировки, математика, программирование, кафедры, преподаватели, стипендии, корпуса, ремонт, айти-этаж, лаборатории, подача заявления, дни открытых дверей, чат приемки, чаты меганаправлений, языки программирования, баллы на общежитие, сертификаты на общежитие, бви, магистратура, военная кафедра, культурные мероприятия, спортивные секции, спортивные соревнования'.split(', ') + ['мусор']
def tag_query(query):
    with open('../research/prompts/tagging_prompt.md', 'r', encoding='utf-8') as f:
        system_prompt = f.read()
    return model.run(system_prompt + "\n\n Входной запрос:\n" + query).text
tag_query('''Маник сделала, крутой капец
''')

'мусор\n'

Слияние в один датафрейм

In [16]:
result = {'source': [], 'chunk_text': []}
result['source'] += list(pd.read_csv('../data/support/final_df.csv')['root_global_id'])
result['chunk_text'] += list(pd.read_csv('../data/support/final_df.csv')['dialog_plain'])
result['source'] += list(pd.read_csv('../data/support/pages_mai.csv')['url'])
result['chunk_text'] += list(pd.read_csv('../data/support/pages_mai.csv')['content'])
result['source'] += list(pd.read_csv('../data/support/pdf_df.csv')['doc_name'])
result['chunk_text'] += list(pd.read_csv('../data/support/pdf_df.csv')['dialog_plain'])
result['source'] += list(pd.read_csv('../data/support/vk_chunks_df.csv')['doc_name'])
result['chunk_text'] += list(pd.read_csv('../data/support/vk_chunks_df.csv')['dialog_plain'])
result_df = pd.DataFrame(result)


In [17]:
result_df

,source,chunk_text
0,chat2021_10,Возможность заселения в одну комнату знакомых/...
1,chat2021_10,Староста Задачи старосты\nПередавать информаци...
2,chat2021_10,Список документов для поступления (бакалавриат...
3,chat2021_10,Добрый день)\nЯляется ли обучение на направлен...
4,chat2021_10,"Доброй ночи, хотелось бы узнать как предоставл..."
...,...,...
4280,ФРЭЛА МАИ,[ФРЭЛА МАИ 💥 В рамках VI Фестиваля радиоэлектр...
4281,ФРЭЛА МАИ,"[ФРЭЛА МАИ Уже завтра в 13:00 Василий Егоров, ..."
4282,ФРЭЛА МАИ,[ФРЭЛА МАИ МАИ приглашает студентов и школьник...
4283,ФРЭЛА МАИ,[ФРЭЛА МАИ 🚀 Попробуй свои силы — участвуй в О...


Обработка диалогов

In [19]:
def add_query(query):
    ans = tag_query(query)

    topics = ans.strip().replace('[', '').replace(']', '').replace('\n\n\u200b', '').split(', ')
    res = []
    for i in topics:
        if i  in valid_topics:
            res.append(i)
    topics = res

    print(f'{query[:15]} -> {topics}')
    return topics

result_df['topics'] = result_df['chunk_text'].apply(add_query)

Возможность зас -> ['общежития', 'айти-этаж', 'кафедры', 'магистратура', 'программирование', 'математика', 'подача заявления']
Староста Задачи -> ['индивидуальные достижения', 'общежития', 'математика']
Список документ -> ['индивидуальные достижения', 'подача заявления', 'магистратура']
Добрый день)
Ял -> ['подача заявления', 'преподаватели']
Доброй ночи, хо -> ['проходной балл', 'общежития', 'количество бюджетных мест', 'баллы на общежитие']
Ок. Если Вы хот -> ['общежития', 'проходной балл', 'количество бюджетных мест', 'бви', 'подача заявления', 'чат приемки']
Здравствуйте! С -> ['подача заявления', 'общежития']
Нашел такой пос -> ['культурные мероприятия', 'спортивные секции']
Привет, студент -> ['спортивные секции', 'корпуса']
Привет! 
Хотел  -> ['стипендии', 'спортивные секции', 'спортивные соревнования', 'программирование', 'кафедры', 'магистратура', 'количество бюджетных мест']
А как вообще ра -> ['проходной балл', 'подача заявления', 'общежития']
Здравствуйте. П -> ['подача зая

Обработка  постов

In [21]:
result_df.to_csv('final_data_with_topics_nd_preprocess.csv')